In [ ]:
!curl -o data_depth_annotated.zip https://s3.eu-central-1.amazonaws.com/avg-kitti/data_depth_annotated.zip
!unzip data_depth_annotated.zip

In [2]:
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
from torch.optim import Adam, SGD, RMSprop
import torch
from torch.utils.data import Dataset, random_split, DataLoader

import matplotlib.pylab as plt
import numpy as np
import cv2
from os import listdir
from os.path import splitext
from pathlib import Path
from PIL import Image
from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
class Encode(nn.Module):
  def __init__(self, channels = (3,16,32,64,96,128,192)):
    super(Encode, self).__init__()
    self.en_blocks = nn.ModuleList([EncodeBlock(channels[i], channels[i+1]) for i in range(len(channels)-1)])

  def forward(self, x):
    enc_features = []
    for block in self.en_blocks:
      x = block(x)
      enc_features.append(x)
    return enc_featuers

class EncodeBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(EncodeBlock, self).__init__()
    down_block1 = nn.Conv2d(in_channels, out_channels, kernel_size= 3, stride = 2, padding = 1 )
    down_block2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.down_block = nn.Sequential(down_block1, nn.LeakyReLU(), down_block2, nn.LeakyReLU())
  
  def forward(self,x):
    return self.down_block(x)

class Decoder(nn.Module):
  def __init__(self, channels = (16,32,64,96,128,192)):
    super(Decoder, self).__init__()
    self.channels = channels
    self.conv_dec = [convDisp(i) for i in channels]
    self.disp = nn.Sigmoid()
    self.deconv = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=2, stride=2)

  def convDisp(self, in_channels):
    chnls = [in_channels] + [96,64,32,8]
    layers = []
    for i in range(len(chnls)-1):
      layers += [nn.Conv2d(chnls[i], chnls[i+1], kernel_size = 3, stride = 1, padding = 1), nn.LeakyReLU()]
    layers.pop()
    return nn.Sequential(*layers)

  def forward(self, enc_features):
    feat = enc_features[-1]
    disp_list = []
    for i in range(len(enc_features),1, -1):
      featb = self.conv_dec[i](feat)
      disp_ = self.disp(featb)
      featb = self.deconv(featb)
      disp_list.append(disp_)
      feat = torch.cat((enc_features[i-2], featb),1)
    featb = self.conv_dec[0](feat)
    disp_ = self.disp(featb)
    disp_list.append(disp_)
    return disp_list.reverse()
  
class DepthModel(nn.Module):
  def __init__(self):
    super(DepthModel, self).__init__()
    self.encode_ = Encode()
    self.decode_ = Decode()

  def forward(self, x):
    # feat = self.encode_(x)
    # disp = self.decode_(feat)
    # return disp
    return self.decode_(self.encode_(x))
